In [1]:
#Finding Native interactions from PDB file

import numpy as np
import pandas as pd
import math

"""Function to covert pdb to DataFrame"""
def pdb_df2 (p):
    n = len(p)      #Provide just the trimmed PDB file
    y = []
    atm_no = []
    res = []
    x_cord = []
    y_cord = []
    z_cord = []

    for i in range(n):
        y = p[i].split()
        atm_no.append(y[1])
        res.append(y[4])
        x_cord.append(y[5])
        y_cord.append(y[6])
        z_cord.append(y[7])
        
    atm_no = pd.DataFrame(data = atm_no)
    res = pd.DataFrame(data = res)
    x_cord = pd.DataFrame(data = x_cord)
    y_cord = pd.DataFrame(data = y_cord)
    z_cord = pd.DataFrame(data = z_cord)
    
    data = pd.DataFrame()
    data["atm_no"] = atm_no[0].values
    data["res"] = res[0].values
    data["X"] = x_cord[0].values
    data["Y"] = y_cord[0].values
    data["Z"] = z_cord[0].values
    return (data)


In [2]:
"""Finding distance between contact pairs 3 residues apart"""

def native_dist (data):
    atm1 = []
    atm2 = []
    r0 = []

    grp = data.groupby("res")

    for i in range(1,len(grp)-2):    #Loop for each residue of the protein
        ab = i+3
        start = grp.get_group(str(ab)).iloc[:,0]

        for j in range(int(start.iloc[start.size-1])+1,len(data)):  #For each atom 3 residues apart
            n = grp.get_group(str(i)).iloc[:,0]

            for m in range(len(n)):    #For distance between each residue each atom of residue i and atom j
                nt = int(n.iloc[m])

                x = (float(data.iloc[j-1,2]) - float(data.iloc[nt-1,2]))
                y = (float(data.iloc[j-1,3]) - float(data.iloc[nt-1,3]))
                z = (float(data.iloc[j-1,4]) - float(data.iloc[nt-1,4]))
                dist = math.sqrt(x**2 + y**2 + z**2)
                atm1.append(nt)
                atm2.append(j)
                r0.append(dist)

    """List of interactions between atoms"""
    atm1 = pd.DataFrame(data=atm1)
    atm2 = pd.DataFrame(data=atm2)
    r0 = pd.DataFrame(data=r0)

    contacts = pd.DataFrame()
    contacts["i"] = atm1[0].values
    contacts["j"] = atm2[0].values
    contacts["r0"] = r0[0].values
    
    return (contacts)


In [3]:
"""Selecting contacts <4.5 nm in distance"""
def compare (contacts):
    ai = []
    aj = []
    res = []
    dist_con = []

    for i in range(len(contacts)):
        if contacts.iloc[i,2] <= 4.5:
            ai.append(contacts.iloc[i,0])
            aj.append(contacts.iloc[i,1])
            dist_con.append(contacts.iloc[i,2])

    ai = pd.DataFrame(data=ai)
    aj = pd.DataFrame(data=aj)
    dist_con = pd.DataFrame(data=dist_con)

    closer = pd.DataFrame()
    closer["i"] = ai[0].values
    closer["j"] = aj[0].values
    closer["r0"] = dist_con[0].values
    
    return (closer)

In [4]:
gb_pdb = open ("mod.pdb", "r")
pdb = gb_pdb.readlines()       #Reading the PDB file

data = pdb_df2(pdb)            #Converting pdb file to dataframe
contacts = native_dist(data)   #Finding the native contact distances
closer = compare (contacts)    #Finding interactions <4.5 A apart

string = []        #Writing in the Contacts file
end = len(closer)
for i in range(0,end):
    a = str(closer.iloc[i,0]) + "\t" + str(closer.iloc[i,1]) + "\t" + str(closer.iloc[i,2]) + "\n"
    string.append(a)

cont = open("Contacts.txt","w")
cont.writelines(string)
cont.close()
gb_pdb.close()